In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from scipy.special import comb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def bernstein(i, n, x):
    return comb(n, i, exact=True) * (x**i) * ((1-x)**(n-i))

def solve_pde(pde_type, nt, nu, Mt):
    model = gp.Model("PDE_solver")
    model.setParam('NumericFocus', 3)

    p = model.addVars([(i, j) for i in range(nt+1) for j in range(nu+1)], 
                      lb=-GRB.INFINITY, ub=GRB.INFINITY, name="p")

    if pde_type == 'a':
        model.addConstrs((p[i, 0] == (i/nt)**2 for i in range(nt+1)), "boundary_condition")
    else:
        for i in range(nt+1):
            model.addConstr(p[i, 0] == 0, f"boundary_u0_{i}")
            model.addConstr(p[i, nu] == 0, f"boundary_u1_{i}")
        for j in range(nu+1):
            model.addConstr(p[0, j] == 6 * np.sin(np.pi * j / nu), f"initial_condition_{j}")

    t_points = np.linspace(0, 1, Mt)
    u_points = np.linspace(0, 1, Mt)

    obj = 0
    for t in t_points:
        for u in u_points:
            f = sum(p[i,j] * bernstein(i, nt, t) * bernstein(j, nu, u) 
                    for i in range(nt+1) for j in range(nu+1))
            
            f_t = nt * sum((p[i+1,j] - p[i,j]) * bernstein(i, nt-1, t) * bernstein(j, nu, u)
                           for i in range(nt) for j in range(nu+1))
            
            f_u = nu * sum((p[i,j+1] - p[i,j]) * bernstein(i, nt, t) * bernstein(j, nu-1, u)
                           for i in range(nt+1) for j in range(nu))
            
            if pde_type == 'a':
                g = f_t + f_u + 2
            else:
                f_uu = nu * (nu-1) * sum(p[i,j] * bernstein(i, nt, t) * 
                                         (bernstein(j+2, nu, u) - 2*bernstein(j+1, nu, u) + bernstein(j, nu, u))
                                         for i in range(nt+1) for j in range(nu-1))
                g = f_t - 4 * f_uu
            obj += g * g

    model.setObjective(obj, GRB.MINIMIZE)
    model.optimize()

    control_points = np.array([[p[i,j].X for j in range(nu+1)] for i in range(nt+1)])
    return control_points

def plot_solution(control_points, pde_type):
    nt, nu = control_points.shape
    nt -= 1
    nu -= 1
    t_plot, u_plot = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
    
    def bezier_surface(t, u):
        return sum(control_points[i, j] * bernstein(i, nt, t) * bernstein(j, nu, u)
                   for i in range(nt+1) for j in range(nu+1))
    
    f_plot = np.vectorize(bezier_surface)(t_plot, u_plot)
    
    if pde_type == 'a':
        title = 'f_t(t,u) + f_u(t,u) + 2 = 0; f(t,0) = t^2'
        f_analytical = 2*u_plot + (t_plot - u_plot)**2
    else:
        title = 'f_t(t,u) - 4f_uu(t,u) = 0; f(0,u) = 6sin(πu), f(t,0) = 0, f(t,1) = 0'
        f_analytical = 6 * np.sin(np.pi * u_plot) * np.exp(-4 * np.pi**2 * t_plot)

    fig = plt.figure(figsize=(20, 10))
    ax1 = fig.add_subplot(121, projection='3d')
    surf1 = ax1.plot_surface(t_plot, u_plot, f_plot, cmap='viridis')
    ax1.set_xlabel('t')
    ax1.set_ylabel('u')
    ax1.set_zlabel('f(t,u)')
    ax1.set_title('Bézier Surface Solution')
    fig.colorbar(surf1, ax=ax1, shrink=0.5, aspect=5)

    ax2 = fig.add_subplot(122, projection='3d')
    surf2 = ax2.plot_surface(t_plot, u_plot, f_analytical, cmap='viridis')
    ax2.set_xlabel('t')
    ax2.set_ylabel('u')
    ax2.set_zlabel('f(t,u)')
    ax2.set_title('Analytical Solution')
    fig.colorbar(surf2, ax=ax2, shrink=0.5, aspect=5)

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

    print(f"Control points shape: {control_points.shape}")
    print(f"Max Bézier surface value: {np.max(f_plot)}")
    print(f"Min Bézier surface value: {np.min(f_plot)}")
    print(f"Max analytical surface value: {np.max(f_analytical)}")
    print(f"Min analytical surface value: {np.min(f_analytical)}")
    print(f"Max absolute difference: {np.max(np.abs(f_plot - f_analytical))}")

# Solve and plot for each PDE type
pde_types = ['a', 'b', 'c', 'd']
for pde_type in pde_types:
    if pde_type == 'a':
        nt, nu, Mt = 3, 3, 17  # 4x4 control points, 17 test points
    elif pde_type == 'b':
        nt, nu, Mt = 2, 2, 10  # 3x3 control points, 10 test points
    elif pde_type == 'c':
        nt, nu, Mt = 5, 5, 37  # 6x6 control points, 37 test points
    #else:  # 'd'
        #nt, nu, Mt = 10, 10, 122  # 11x11 control points, 122 test points

    control_points = solve_pde(pde_type, nt, nu, Mt)
    plot_solution(control_points, pde_type)
    print(f"Solved PDE type {pde_type}")

In [ ]:
et parameter NumericFocus to value 3
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 22.6.0 22G74)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4 rows, 16 columns and 4 nonzeros
Model fingerprint: 0xd62634ee
Model has 136 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 6e+02]
  QObjective range [7e-01, 7e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+00]
Presolve removed 4 rows and 4 columns
Presolve time: 0.02s
Presolved: 0 rows, 12 columns, 0 nonzeros
Presolved model has 78 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 23
 AA' NZ     : 5.500e+01
...

Barrier solved model in 8 iterations and 0.07 seconds (0.00 work units)
Optimal objective -3.87672117e-11

Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

Control points shape: (4, 4)
Max Bézier surface value: 1.0
Min Bézier surface value: -2.041662396407697
Max analytical surface value: 3.0
Min analytical surface value: 0.0
Max absolute difference: 4.666666651162887
Solved PDE type a
Set parameter NumericFocus to value 3
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 22.6.0 22G74)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 9 columns and 9 nonzeros
Model fingerprint: 0x83cc0ec5
Model has 45 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [9e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-16, 6e+00]
Presolve removed 9 rows and 7 columns
Presolve time: 0.01s
Presolved: 0 rows, 2 columns, 0 nonzeros
...

Barrier solved model in 8 iterations and 0.07 seconds (0.00 work units)
Optimal objective 1.05728759e-11

Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...
Control points shape: (3, 3)
Max Bézier surface value: 2.999693908784818
Min Bézier surface value: 0.0
Max analytical surface value: 5.99924476604325
Min analytical surface value: 0.0
Max absolute difference: 2.9996938851254615
Solved PDE type b
Set parameter NumericFocus to value 3
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 22.6.0 22G74)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 18 rows, 36 columns and 18 nonzeros
Model fingerprint: 0x57dd7766
Model has 666 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+00, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-16, 6e+00]
Presolve removed 18 rows and 16 columns
Presolve time: 0.01s
Presolved: 0 rows, 20 columns, 0 nonzeros
...

Barrier solved model in 8 iterations and 0.02 seconds (0.00 work units)
Optimal objective 1.01363381e+06

Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

Control points shape: (6, 6)
Max Bézier surface value: 4.66805640662936
Min Bézier surface value: -0.1446787873270405
Max analytical surface value: 5.99924476604325
Min analytical surface value: 0.0
Max absolute difference: 3.0615364528757705
Solved PDE type c
Set parameter NumericFocus to value 3
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 22.6.0 22G74)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 18 rows, 36 columns and 18 nonzeros
Model fingerprint: 0x57dd7766
Model has 666 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+00, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-16, 6e+00]
Presolve removed 18 rows and 16 columns
Presolve time: 0.01s
Presolved: 0 rows, 20 columns, 0 nonzeros
...

Barrier solved model in 8 iterations and 0.02 seconds (0.00 work units)
Optimal objective 1.01363381e+06

Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

Control points shape: (6, 6)
Max Bézier surface value: 4.66805640662936
Min Bézier surface value: -0.1446787873270405
Max analytical surface value: 5.99924476604325
Min analytical surface value: 0.0
Max absolute difference: 3.0615364528757705
Solved PDE type d